In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
import pickle
import datetime

import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Flatten,MaxPool2D,BatchNormalization,GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from keras.models import load_model

In [ ]:
ROOT_DIR = _YOUR_DIR_
TRAIN_PATH = os.path.join(ROOT_DIR, 'dataset/train')
TEST_PATH = os.path.join(ROOT_DIR, 'dataset/test')

# Image Data Generators

In [ ]:
batch_size = 32
img_height = 480
img_width = 480
target_size = (img_height, img_width)

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical', 
        color_mode='rgb',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
    TRAIN_PATH, 
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical', 
    color_mode='rgb',
    subset='validation') 


test_generator = test_datagen.flow_from_directory(
        TEST_PATH,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical', 
        color_mode='rgb',
        shuffle=False)

Found 24104 images belonging to 3 classes.
Found 6026 images belonging to 3 classes.
Found 400 images belonging to 3 classes.


In [ ]:
counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

In [ ]:
print(train_generator.class_indices)
print(counter)
print(class_weights)

{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}
Counter({0: 13192, 1: 6468, 2: 4444})
{0: 1.0, 1: 2.0395794681508965, 2: 2.9684968496849686}


# Create Model

In [ ]:
base_model = VGG16(include_top=False, weights="imagenet", input_shape=(img_height, img_width, 3))

for layer in base_model.layers[:(len(base_model.layers) // 3 * 2)]:
    layer.trainable = False

for i, layer in enumerate(base_model.layers):
    print(i, layer.name, "-", layer.trainable)

58900480/58889256 [==============================] - 0s 0us/step
0 input_1 - False
1 block1_conv1 - False
2 block1_conv2 - False
3 block1_pool - False
4 block2_conv1 - False
5 block2_conv2 - False
6 block2_pool - False
7 block3_conv1 - False
8 block3_conv2 - False
9 block3_conv3 - False
10 block3_pool - False
11 block4_conv1 - False
12 block4_conv2 - True
13 block4_conv3 - True
14 block4_pool - True
15 block5_conv1 - True
16 block5_conv2 - True
17 block5_conv3 - True
18 block5_pool - True


In [ ]:
model = tf.keras.Sequential([
    base_model, 
    tf.keras.layers.GlobalAveragePooling2D(), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 15, 15, 512)       14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 3

# Callbacks

In [ ]:
patience = 1
stop_patience = 3
factor = 0.5
callbacks = [
    ModelCheckpoint(os.path.join(ROOT_DIR, 'model/vgg.h5'), save_best_only=True, save_weights_only=True, verbose=1),
    # EarlyStopping(patience=stop_patience, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, verbose=1)
]

# Model Training

In [ ]:
epochs = 30
print(datetime.datetime.now())
history = model.fit(train_generator, validation_data=validation_generator, class_weight=class_weights, epochs=epochs, callbacks=callbacks, verbose=1)
print(datetime.datetime.now())

2022-09-19 11:45:42.579630
Epoch 1/30
754/754 [==============================] - ETA: 0s - loss: 0.9537 - accuracy: 0.7713 
Epoch 1: val_loss improved from inf to 19326.79102, saving model to /content/drive/MyDrive/DTA/Detect-COVID19/model/vgg.h5
754/754 [==============================] - 15577s 21s/step - loss: 0.9537 - accuracy: 0.7713 - val_loss: 19326.7910 - val_accuracy: 0.2683 - lr: 0.0010
Epoch 2/30
754/754 [==============================] - ETA: 0s - loss: 0.5913 - accuracy: 0.8808
Epoch 2: val_loss improved from 19326.79102 to 2.12183, saving model to /content/drive/MyDrive/DTA/Detect-COVID19/model/vgg.h5
754/754 [==============================] - 1604s 2s/step - loss: 0.5913 - accuracy: 0.8808 - val_loss: 2.1218 - val_accuracy: 0.4544 - lr: 0.0010
Epoch 3/30
754/754 [==============================] - ETA: 0s - loss: 0.4897 - accuracy: 0.9036
Epoch 3: val_loss improved from 2.12183 to 1.37114, saving model to /content/drive/MyDrive/DTA/Detect-COVID19/model/vgg.h5
754/754 [====

In [ ]:
with open(os.path.join(ROOT_DIR, 'history_dict_vgg.pkl'), 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
pickle_in = open(os.path.join(ROOT_DIR, 'history_dict_resnet.pkl'), "rb")
dict_log = pickle.load(pickle_in)